## Import

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
from functools import reduce

import numpy as np
import pandas as pd
import re, time
import requests
import pickle
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
import os
os.getcwd()

'C:\\Users\\PC0\\Documents\\GitHub\\Stock-price-prediction\\ipynb'

In [3]:
out_data_path = 'C:\\Users\\PC0\\Documents\\datasets\\stock_price\\'
data_path = 'C:\\Users\\PC0\\Documents\\GitHub\\Stock-price-prediction\\datasets\\'

* crawling

In [4]:
options = webdriver.ChromeOptions()

options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36")

driver = webdriver.Chrome("C:\\Users\\PC0\\Desktop\\chromedriver.exe", options=options)

* agricultural 

In [70]:
start_day = "2012/01/01"

In [71]:
urls = ['https://kr.investing.com/commodities/softs',
       'https://kr.investing.com/commodities/energy']

In [72]:
for idx_1, url in enumerate(urls):
    driver.get(url)
    time.sleep(3)
    
    temp_names = driver.find_elements_by_css_selector("table#cross_rate_1 tbody tr td.bold.left.noWrap.elp.plusIconTd a")
    names = [re.sub(" ","_",x.text) for x in temp_names]
    urls = [x.get_attribute("href") + "-historical-data" for x in temp_names]
    index_names = [(x.split("/")[-1]).split("-historical")[0] for x in urls]
    
    if idx_1 == 0:
        symbol_df = pd.DataFrame({"symbol":index_names,
                                 "name":names,
                                 "url":urls})
    else:
        temp_df = pd.DataFrame({"symbol":index_names,
                                 "name":names,
                                 "url":urls})
        symbol_df = symbol_df.append(temp_df)
        
    
    # inner loop
    index_list = []
    start_idx = 0

    for idx, url in tqdm(enumerate(urls[start_idx:])):
        driver.get(url)
        time.sleep(5)

        # 날짜 입력란
        tmp_date = driver.find_element_by_css_selector("div#widgetFieldDateRange")
        tmp_date.click()
        driver.implicitly_wait(3)

        tmp_input = driver.find_element_by_css_selector("input#startDate")
        time.sleep(0.5)

        # 기존 날짜 지우기
        for _ in range(11) :
            tmp_input.send_keys(Keys.BACKSPACE)
            tmp_input = driver.find_element_by_css_selector("input#startDate")
            time.sleep(0.5)

        tmp_input.send_keys(start_day)

        # 요청 버튼
        request_btn = driver.find_element_by_css_selector("a#applyBtn")
        request_btn.click()
        time.sleep(5)

        tables = driver.find_elements_by_css_selector("table#curr_table")

        values = tables[0].text.split("\n")[1:]

        values = [re.sub("[^0-9a-zA-Z.\\- ]","",x) for x in values]

        name = names[idx]

        index_dict = {"date":[],
                     "{}_close".format(name):[],
                     "{}_vol".format(name):[],
                     "{}_var".format(name):[]}

        for val in values:
            year, month, day, close, _, _, _, volume, variation= val.split(" ")
            date = "-".join([year, month, day])
            index_dict['date'].append(date)
            index_dict['{}_close'.format(name)].append(close)
            index_dict['{}_vol'.format(name)].append(volume)
            index_dict['{}_var'.format(name)].append(variation)

        index_list.append(pd.DataFrame.from_dict(index_dict, orient = "index").T)
        start_idx += 1
        
        if idx_1 == 0:
            index_df = reduce(lambda  left,right: pd.merge(left,right,on=['date'], how='inner'), index_list)
        else :
            temp_df = reduce(lambda  left,right: pd.merge(left,right,on=['date'], how='inner'), index_list)
            index_df = pd.merge(index_df, temp_df, on = "date", how = "inner")

9it [03:19, 22.19s/it]
7it [02:35, 22.25s/it]


In [73]:
symbol_df.to_csv(data_path + "materials_symbol_df.csv", index = False)
index_df.to_csv(data_path + "materials__df.csv", index = False)